In [35]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [36]:
from langchain.llms import OpenAI
llm = OpenAI()

In [ ]:
from langchain import PromptTemplate
# Prompt đơn giản được sử dụng
custom_prompt_template = """Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.

Context: {context}
Question: {question}

"""
prompt = PromptTemplate(template=custom_prompt_template, input_variables=["text"])
prompt


PromptTemplate(input_variables=['context', 'question'], template='Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.\nNếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.\nTất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt\n\nContext: {context}\nQuestion: {question}\n\n')

In [38]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_path = "./stsv.txt"
with open(file_path, "r", encoding="utf-8") as file:
    explanation = file.read()
print(explanation)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
texts = text_splitter.create_documents([explanation])

I. Giới thiệu về Trường Khoa Công nghệ Thông tin là đơn vị đào tạo thành viên thuộc Đại học Thái Nguyên được thành lập ngày 14 tháng 11 năm 2001 theo Quyết định số 6946QĐ-BGD ĐT-TCCB của Bộ trưởng Bộ Giáo dục và Đào tạo. Sau 10 năm xây dựng và phát triển, ngày 30 tháng 3 năm 2011, Thủ tướng Chính phủ ký Quyết định số 468QĐ-TTg thành lập Trường Đại học Công nghệ Thông tin và Truyền thông trên cơ sở nâng cấp Khoa Công nghệ Thông tin thuộc Đại học Thái Nguyên. II. Tầm nhìn Trường Đại học Công nghệ Thông tin và Truyền thông trở thành Trường đại học ứng dụng, đa ngành, trên nền tảng số hàng đầu trong hệ thống giáo dục đại học Việt Nam. III. Sứ mạng Đào tạo nguồn nhân lực trình độ đại học, sau đại học bồi dưỡng ngắn hạn nghiên cứu khoa học và chuyển giao công nghệ đáp ứng nhu cầu của thị trường lao động và phù hợp với Chiến lực Quốc gia về cách mạng công nghiệp lần thứ tư và Chương trình Chuyển đổi số Quốc gia, phục vụ phát triển kinh tế - văn hóa – xã hội của đất nước. IV. Địa chỉ liên hệ T

In [40]:
texts[0].page_content

'I. Giới thiệu về Trường Khoa Công nghệ Thông tin là đơn vị đào tạo thành viên thuộc Đại học Thái Nguyên được thành lập ngày 14 tháng 11 năm 2001 theo Quyết định số 6946QĐ-BGD ĐT-TCCB của Bộ trưởng Bộ Giáo dục và Đào tạo. Sau 10 năm xây dựng và phát triển, ngày 30 tháng 3 năm 2011, Thủ tướng Chính phủ ký Quyết định số 468QĐ-TTg thành lập Trường Đại học Công nghệ Thông tin và Truyền thông trên cơ sở nâng cấp Khoa Công nghệ Thông tin thuộc Đại học Thái Nguyên. II. Tầm nhìn Trường Đại học Công nghệ'

In [41]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [42]:
query_result = embeddings.embed_query(texts[0].page_content)
query_result

[-0.003442861837598431,
 -0.014014172348618447,
 0.008865847582340904,
 -0.04864718807619117,
 -0.024259720214970148,
 -0.004471249109500478,
 -0.008271810968991199,
 0.0007345623537036499,
 -0.02438746972386605,
 -0.01194462192260225,
 0.007083735879646764,
 0.018804796627342444,
 0.007971598226102064,
 -0.02446411942920359,
 -0.013337097157502639,
 -0.0039219243586030815,
 0.006154354709969667,
 -0.00422213663583096,
 0.019213595055809326,
 -0.007045411026977993,
 -0.001711849589216709,
 0.01508727194863406,
 -0.022637294932734628,
 0.023748720316741523,
 0.014205796611962298,
 0.0010012403848463542,
 0.007179548476979945,
 -0.01925192083980061,
 0.008923335327005316,
 0.003944280289829237,
 0.025128418738107297,
 0.005598642483627477,
 -0.035616693465328746,
 -0.0185876215308969,
 0.0004746709861172119,
 -0.01435909695395989,
 -0.01011141041634975,
 -0.017961646143339453,
 0.035131243468879296,
 0.0074350484260942585,
 0.04586223946903542,
 0.013107147110167505,
 0.00825264807699555

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os

# Khởi tạo Pinecone với API Key và môi trường
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


In [ ]:

import time
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore

index_name = "langchain-stsv"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
uuids = [str(uuid4()) for _ in range(len(texts))]
vector_store.add_documents(documents=texts, ids=uuids)

['4f067af7-8072-4d05-9610-a68a44e4bc44',
 '7cbd22ef-1c71-43b9-a6b4-d8c1d3b151d2',
 'a76ac192-b739-4981-b239-d8a5ac22a11b',
 '25a4e6b2-7d6c-425a-aea6-42dc2dc96856',
 'afefa5e4-df84-4cfd-bba7-fdae32682f1a',
 '12b8d493-0718-4d6f-b091-36c124343798',
 'c10bc959-6d9d-4953-a847-56b22d28e93d',
 'ae6c7027-4201-41c6-b296-a67eb41820e1',
 '0f135aea-9cab-4780-bbe6-296882a6056b',
 '768fc9e5-053c-4535-b5cf-22ed886d5ccc',
 '71752b6c-ccfa-46e9-b5af-1920cd217180',
 '24948007-2278-4c00-adb2-0eefcaf3b57e',
 '909f3c4c-7101-4700-bb48-072f55b3c45d',
 '85ffe96c-7350-42a4-bc8d-41d88c17cc60',
 'b5d222fe-37a5-4f86-b09b-48bfece523ac',
 'a6ae599f-4225-4bbc-be96-b698db06770a',
 '32f68a3b-d587-407f-bebb-28212340b858',
 'e79edf24-6165-4278-bf0c-6da9b4923a29',
 '6eb0c099-5c96-4fb2-8333-4a9d685835fc',
 '33bf042b-0f6e-485c-8778-492cc5413438',
 '2e74ac42-588e-41d1-8a47-08d88c91d4fd',
 '1a596699-d12d-44ea-874b-19e3c7dcce78',
 '3cb40c37-e7a6-4964-ab66-54e62f0ab325',
 '255327bc-bbe8-41bf-bf0d-358d52cd7ea3',
 'dab27467-1ac0-

In [45]:
query = "ICTU ở đâu?"

result = vector_store.similarity_search(query)

context = " ".join([doc.page_content for doc in result])


In [46]:
chain.run(question = query,context=context)

'Trường Đại học Công nghệ Thông tin Truyền thông (ICTU) nằm tại xã Quyết Thắng, thành phố Thái Nguyên, tỉnh Thái Nguyên.'

In [47]:
from transformers import pipeline

# Khởi tạo pipeline cho nhiệm vụ hỏi đáp
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
# Sử dụng pipeline để trả lời câu hỏi
result = qa_pipeline(question=query, context=context)

# In kết quả
print(result['answer'])

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


Đảng ủy Party Committee Ban Giám Hiệu
